# Scrape Realtor.com Listings

In [1]:
import pandas as pd
import requests
from time import sleep
import concurrent.futures
import json
from datetime import datetime
import re

## Setup Headers and Other Constants

In [2]:
URL = "https://www.realtor.com/api/v1/hulk_main_srp"

QUERYSTRING = {"client_id": "rdc-x", "schema": "vesta"}

HEADERS = {
    "authority": "www.realtor.com",
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.5",
    "content-type": "application/json",
    "origin": "https://www.realtor.com",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

QUERY = """
    query ConsumerSearchMainQuery(
        $query: HomeSearchCriteria!
        $limit: Int
        $offset: Int
        $sort: [SearchAPISort]
        $sort_type: SearchSortType
        $client_data: JSON
        $bucket: SearchAPIBucket
        ) {
        home_search: home_search(
            query: $query
            sort: $sort
            limit: $limit
            offset: $offset
            sort_type: $sort_type
            client_data: $client_data
            bucket: $bucket
        ) {
            count
            total
            results {
            property_id
            list_price
            primary
            rent_to_own {
                rent
                right_to_purchase
                provider
            }
            primary_photo(https: true) {
                href
            }
            source {
                id
                agents {
                office_name
                }
                type
                spec_id
                plan_id
            }
            community {
                property_id
                permalink
                description {
                name
                }
                advertisers {
                office {
                    hours
                    phones {
                    type
                    number
                    }
                }
                builder {
                    fulfillment_id
                }
                }
            }
            products {
                brand_name
                products
            }
            listing_id
            matterport
            virtual_tours {
                href
                type
            }
            status
            permalink
            price_reduced_amount
            other_listings {
                rdc {
                listing_id
                status
                listing_key
                primary
                }
            }
            description {
                beds
                baths
                baths_full
                baths_half
                baths_1qtr
                baths_3qtr
                garage
                stories
                type
                sub_type
                lot_sqft
                sqft
                year_built
                sold_price
                sold_date
                name
            }
            location {
                street_view_url
                address {
                line
                postal_code
                state
                state_code
                city
                coordinate {
                    lat
                    lon
                }
                }
                county {
                name
                fips_code
                }
            }
            tax_record {
                public_record_id
            }
            lead_attributes {
                show_contact_an_agent
                opcity_lead_attributes {
                cashback_enabled
                flip_the_market_enabled
                }
                lead_type
                ready_connect_mortgage {
                show_contact_a_lender
                show_veterans_united
                }
            }
            open_houses {
                start_date
                end_date
                description
                methods
                time_zone
                dst
            }
            flags {
                is_coming_soon
                is_pending
                is_foreclosure
                is_contingent
                is_new_construction
                is_new_listing(days: 14)
                is_price_reduced(days: 30)
                is_plan
                is_subdivision
            }
            list_date
            last_update_date
            coming_soon_date
            photos(limit: 2, https: true) {
                href
            }
            tags
            branding {
                type
                photo
                name
            }
            }
        }
    }
"""

## Pull from Realtor

In [3]:
results = []

offset = 0
# total gets updated, just need a value greater than offset for first iteration
total = offset + 1

# for the purpose of this example, creating a max
max_results = 200

while offset < total:

    print(f"handling offset {offset} in a total of {total}      ", end='\r')

    payload = {
        "query": QUERY,
        "variables": {
            "query": {
                "status": ["for_sale", "ready_to_build"],
                "primary": True,
                "search_location": {"location": "Burke, VA"}
            },
            "limit": 42,
            "offset": offset,
            "sort_type": "relevant",
            "by_prop_type": ["home"]
        },
        "operationName": "ConsumerSearchMainQuery",
        "callfrom": "SRP",
        "nrQueryType": "MAIN_SRP",
        "isClient": True,
    }

    response = requests.request(
        "POST", URL, json=payload, headers=HEADERS, params=QUERYSTRING)

    if response.status_code != 200:
        raise ValueError(f"Bad status code on response: {response.status_code}")

    try:
        data = response.json()['data']['home_search']
    except:
        print("Failed to read data, something went wrong with the request")
        raise

    total = data['total']

    response_results = data['results']
    offset += len(response_results)

    results += response_results

    if max_results and offset >= max_results:
        print("\n")
        print(f"Hit max: {max_results}")
        break

print("Done!                                            ")

Done!                                            


## Parsing Data

We now have our listings data, my preference is to extract the parts I'm interested in and structure them within a pandas `DataFrame`

In [4]:
def parse_flags(flags):
    # flags example value
    # {'is_coming_soon': None, 'is_new_listing': False, 'is_price_reduced': None, 'is_foreclosure': None, 'is_new_construction': None, 'is_pending': True, 'is_contingent': None}

    status = []
    if flags.get('is_coming_soon') is True:
        status.append("coming soon")
    if flags.get('is_new_listing') is True:
        status.append("new listing")
    if flags.get('is_price_reduced') is True:
        status.append("price reduced")
    if flags.get('is_foreclosure') is True:
        status.append("foreclosure")
    if flags.get('is_new_construction') is True:
        status.append("new construction")
    if flags.get('is_pending') is True:
        status.append("pending")
    if flags.get('is_contingent') is True:
        status.append("contingent")

    return ", ".join(status)

In [5]:
df = pd.DataFrame()

for result in results:

    if result['location']['address'].get('coordinate'):
        lat = result['location']['address'].get('coordinate', {}).get('lat')
        lon = result['location']['address'].get('coordinate', {}).get('lon')
    else:
        lat, lon = None, None

    df = pd.concat([df, pd.DataFrame([{
            'id': result['property_id'],
            'list date': pd.to_datetime(result['list_date']),
            'status': result['status'],
            'flags': parse_flags(result['flags']),
            'home type': result['description']['type'],
            'year built': result['description']['year_built'],
            'price': result['list_price'],
            'hoa fee': result.get('hoa', {}).get('fee', None),
            'beds': result['description']['beds'],
            'baths': result['description']['baths'],
            'interior sqft': result['description']['sqft'],
            'lot sqft': result['description']['lot_sqft'],
            'address': f"{result['location']['address']['line']} {result['location']['address']['city']}, {result['location']['address']['state_code']} {result['location']['address']['postal_code']}",
            'addresss line': result['location']['address']['line'],
            'city': result['location']['address']['city'],
            'state': result['location']['address']['state_code'],
            'zipcode': result['location']['address']['postal_code'],
            'latitude': lat,
            'longitude': lon,
            'url': f"https://www.realtor.com/realestateandhomes-detail/{result['permalink']}",
        }])])

print(df.shape)
df.head()

(50, 20)


,id,list date,status,flags,home type,year built,price,hoa fee,beds,baths,interior sqft,lot sqft,address,addresss line,city,state,zipcode,latitude,longitude,url
0,5057900396,2023-11-17 02:04:50+00:00,for_sale,new listing,single_family,1977,850000,None,4,4,3528,10498,"8902 Grass Valley Ct Springfield, VA 22153",8902 Grass Valley Ct,Springfield,VA,22153,38.759675,-77.254183,https://www.realtor.com/realestateandhomes-det...
0,6795635263,2023-11-17 05:14:02+00:00,for_sale,new listing,single_family,1987,899888,None,4,4,3866,10485,"6054 Woodedge Ct Burke, VA 22015",6054 Woodedge Ct,Burke,VA,22015,38.787709,-77.279857,https://www.realtor.com/realestateandhomes-det...
0,6654012086,2023-11-17 05:14:02+00:00,for_sale,new listing,condos,1974,425000,None,3,3,1460,None,"6448 Birch Leaf Ct Unit 28B Burke, VA 22015",6448 Birch Leaf Ct Unit 28B,Burke,VA,22015,38.776820,-77.261609,https://www.realtor.com/realestateandhomes-det...
0,6635525029,2023-11-10 05:15:07+00:00,for_sale,"new listing, contingent",condos,1981,372500,None,3,2,1079,None,"5804 Cove Landing Rd Apt 202 Burke, VA 22015",5804 Cove Landing Rd Apt 202,Burke,VA,22015,38.795661,-77.306284,https://www.realtor.com/realestateandhomes-det...
0,5957870279,2023-11-09 19:17:11+00:00,for_sale,pending,single_family,1977,784900,None,4,3,2034,16310,"9210 Rockefeller Ln Springfield, VA 22153",9210 Rockefeller Ln,Springfield,VA,22153,38.758991,-77.260197,https://www.realtor.com/realestateandhomes-det...


### Database store

Now that we have the basic dataframe. Lets pop it into a basic SQL Lite database

https://www.geeksforgeeks.org/how-to-convert-pandas-dataframe-into-sql-in-python/#

In [6]:
import sqlalchemy as db

In [7]:
engine = db.create_engine('sqlite:///test.db')

Note that the `if_exists='replace` actually drops the table and replaces all the content.

Only use this code if the dataframe does not exist!

In [24]:
df.to_sql(name='House_Sale', con = engine, if_exists='Fail', index=True)

46

Ok - Lets try and get the data back

In [8]:
conn = engine.connect()
metadata = db.MetaData()

In [9]:
housedata = db.Table('House_Sale', metadata, autoload_with=engine)

In [10]:
query = housedata.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
for r in result:
    print(r)

('6992687417', datetime.datetime(2023, 11, 16, 5, 13, 25), 'for_sale', 'new listing', 'single_family', 1993, 1399900, None, 5, 5, 5931, 26609, '6486 Lake Meadow Dr Burke, VA 22015', '6486 Lake Meadow Dr', 'Burke', 'VA', '22015', 38.778356, -77.289498, 'https://www.realtor.com/realestateandhomes-detail/6486-Lake-Meadow-Dr_Burke_VA_22015_M69926-87417')
('5387462722', datetime.datetime(2023, 11, 15, 18, 15, 48), 'for_sale', 'new listing', 'single_family', 1970, 799888, None, 4, 3, 2200, 9158, '9306 Winbourne Rd Burke, VA 22015', '9306 Winbourne Rd', 'Burke', 'VA', '22015', 38.806969, -77.266108, 'https://www.realtor.com/realestateandhomes-detail/9306-Winbourne-Rd_Burke_VA_22015_M53874-62722')
('6635525029', datetime.datetime(2023, 11, 10, 5, 15, 7), 'for_sale', 'new listing, contingent', 'condos', 1981, 372500, None, 3, 2, 1079, None, '5804 Cove Landing Rd Apt 202 Burke, VA 22015', '5804 Cove Landing Rd Apt 202', 'Burke', 'VA', '22015', 38.795661, -77.306284, 'https://www.realtor.com/real

In [11]:
print(conn.execute(
    db.sql.text("SELECT * FROM House_Sale")
    ).fetchall())

[('6992687417', '2023-11-16 05:13:25.000000', 'for_sale', 'new listing', 'single_family', 1993, 1399900, None, 5, 5, 5931, 26609, '6486 Lake Meadow Dr Burke, VA 22015', '6486 Lake Meadow Dr', 'Burke', 'VA', '22015', 38.778356, -77.289498, 'https://www.realtor.com/realestateandhomes-detail/6486-Lake-Meadow-Dr_Burke_VA_22015_M69926-87417'), ('5387462722', '2023-11-15 18:15:48.000000', 'for_sale', 'new listing', 'single_family', 1970, 799888, None, 4, 3, 2200, 9158, '9306 Winbourne Rd Burke, VA 22015', '9306 Winbourne Rd', 'Burke', 'VA', '22015', 38.806969, -77.266108, 'https://www.realtor.com/realestateandhomes-detail/9306-Winbourne-Rd_Burke_VA_22015_M53874-62722'), ('6635525029', '2023-11-10 05:15:07.000000', 'for_sale', 'new listing, contingent', 'condos', 1981, 372500, None, 3, 2, 1079, None, '5804 Cove Landing Rd Apt 202 Burke, VA 22015', '5804 Cove Landing Rd Apt 202', 'Burke', 'VA', '22015', 38.795661, -77.306284, 'https://www.realtor.com/realestateandhomes-detail/5804-Cove-Landing

### Merging data

Lets implement the situation where we have new records.

In [12]:
ds = pd.read_sql("SELECT * FROM House_Sale", con=engine)

In [15]:
merged_df = pd.merge(ds, df, on=['id', 'status'])
# Update the dataframe in the database with the merged records

In [20]:
df.shape

(50, 20)

In [21]:
df

,id,list date,status,flags,home type,year built,price,hoa fee,beds,baths,interior sqft,lot sqft,address,addresss line,city,state,zipcode,latitude,longitude,url
0,5057900396,2023-11-17 02:04:50+00:00,for_sale,new listing,single_family,1977,850000,None,4,4,3528,10498,"8902 Grass Valley Ct Springfield, VA 22153",8902 Grass Valley Ct,Springfield,VA,22153,38.759675,-77.254183,https://www.realtor.com/realestateandhomes-det...
0,6795635263,2023-11-17 05:14:02+00:00,for_sale,new listing,single_family,1987,899888,None,4,4,3866,10485,"6054 Woodedge Ct Burke, VA 22015",6054 Woodedge Ct,Burke,VA,22015,38.787709,-77.279857,https://www.realtor.com/realestateandhomes-det...
0,6654012086,2023-11-17 05:14:02+00:00,for_sale,new listing,condos,1974,425000,None,3,3,1460,None,"6448 Birch Leaf Ct Unit 28B Burke, VA 22015",6448 Birch Leaf Ct Unit 28B,Burke,VA,22015,38.776820,-77.261609,https://www.realtor.com/realestateandhomes-det...
0,6635525029,2023-11-10 05:15:07+00:00,for_sale,"new listing, contingent",condos,1981,372500,None,3,2,1079,None,"5804 Cove Landing Rd Apt 202 Burke, VA 22015",5804 Cove Landing Rd Apt 202,Burke,VA,22015,38.795661,-77.306284,https://www.realtor.com/realestateandhomes-det...
0,5957870279,2023-11-09 19:17:11+00:00,for_sale,pending,single_family,1977,784900,None,4,3,2034,16310,"9210 Rockefeller Ln Springfield, VA 22153",9210 Rockefeller Ln,Springfield,VA,22153,38.758991,-77.260197,https://www.realtor.com/realestateandhomes-det...
0,6494123889,2023-11-09 16:50:41+00:00,for_sale,"new listing, contingent",townhomes,1980,560000,None,3,3,2023,1550,"5701 Walnut Wood Ln Burke, VA 22015",5701 Walnut Wood Ln,Burke,VA,22015,38.796490,-77.295153,https://www.realtor.com/realestateandhomes-det...
0,5698564405,2023-11-10 05:15:07+00:00,for_sale,new listing,single_family,1981,799000,None,5,3,1501,8675,"10217 Steamboat Landing Ln Burke, VA 22015",10217 Steamboat Landing Ln,Burke,VA,22015,38.786666,-77.309752,https://www.realtor.com/realestateandhomes-det...
0,5959122647,2023-11-07 00:39:17+00:00,for_sale,"new listing, contingent",single_family,1978,649900,None,4,3,1774,2717,"9814 Pebble Weigh Ct Burke, VA 22015",9814 Pebble Weigh Ct,Burke,VA,22015,38.793154,-77.259604,https://www.realtor.com/realestateandhomes-det...
0,5157781713,2023-11-08 17:12:35+00:00,for_sale,pending,single_family,1977,769900,None,4,3,2182,16166,"5624 Signal Point Ct Burke, VA 22015",5624 Signal Point Ct,Burke,VA,22015,38.797391,-77.249788,https://www.realtor.com/realestateandhomes-det...
0,6992687417,2023-11-16 05:13:25+00:00,for_sale,new listing,single_family,1993,1399900,None,5,5,5931,26609,"6486 Lake Meadow Dr Burke, VA 22015",6486 Lake Meadow Dr,Burke,VA,22015,38.778356,-77.289498,https://www.realtor.com/realestateandhomes-det...


In [19]:
merged_df.shape

(46, 38)

In [22]:
merged_df

,id,list date_x,status,flags_x,home type_x,year built_x,price_x,hoa fee_x,beds_x,baths_x,...,interior sqft_y,lot sqft_y,address_y,addresss line_y,city_y,state_y,zipcode_y,latitude_y,longitude_y,url_y
0,6992687417,2023-11-16 05:13:25.000000,for_sale,new listing,single_family,1993,1399900,None,5,5,...,5931,26609,"6486 Lake Meadow Dr Burke, VA 22015",6486 Lake Meadow Dr,Burke,VA,22015,38.778356,-77.289498,https://www.realtor.com/realestateandhomes-det...
1,5387462722,2023-11-15 18:15:48.000000,for_sale,new listing,single_family,1970,799888,None,4,3,...,2200,9158,"9306 Winbourne Rd Burke, VA 22015",9306 Winbourne Rd,Burke,VA,22015,38.806969,-77.266108,https://www.realtor.com/realestateandhomes-det...
2,6635525029,2023-11-10 05:15:07.000000,for_sale,"new listing, contingent",condos,1981,372500,None,3,2,...,1079,None,"5804 Cove Landing Rd Apt 202 Burke, VA 22015",5804 Cove Landing Rd Apt 202,Burke,VA,22015,38.795661,-77.306284,https://www.realtor.com/realestateandhomes-det...
3,5957870279,2023-11-09 19:17:11.000000,for_sale,pending,single_family,1977,784900,None,4,3,...,2034,16310,"9210 Rockefeller Ln Springfield, VA 22153",9210 Rockefeller Ln,Springfield,VA,22153,38.758991,-77.260197,https://www.realtor.com/realestateandhomes-det...
4,5959122647,2023-11-07 00:39:17.000000,for_sale,"new listing, contingent",single_family,1978,649900,None,4,3,...,1774,2717,"9814 Pebble Weigh Ct Burke, VA 22015",9814 Pebble Weigh Ct,Burke,VA,22015,38.793154,-77.259604,https://www.realtor.com/realestateandhomes-det...
5,5698564405,2023-11-10 05:15:07.000000,for_sale,new listing,single_family,1981,799000,None,5,3,...,1501,8675,"10217 Steamboat Landing Ln Burke, VA 22015",10217 Steamboat Landing Ln,Burke,VA,22015,38.786666,-77.309752,https://www.realtor.com/realestateandhomes-det...
6,6494123889,2023-11-09 16:50:41.000000,for_sale,new listing,townhomes,1980,560000,None,3,3,...,2023,1550,"5701 Walnut Wood Ln Burke, VA 22015",5701 Walnut Wood Ln,Burke,VA,22015,38.796490,-77.295153,https://www.realtor.com/realestateandhomes-det...
7,5995152669,2023-10-20 04:03:09.000000,for_sale,"price reduced, contingent",single_family,1982,930000,None,4,4,...,2476,10006,"5603 Bakersville Ln Burke, VA 22015",5603 Bakersville Ln,Burke,VA,22015,38.797937,-77.263053,https://www.realtor.com/realestateandhomes-det...
8,5157781713,2023-11-08 17:12:35.000000,for_sale,pending,single_family,1977,769900,None,4,3,...,2182,16166,"5624 Signal Point Ct Burke, VA 22015",5624 Signal Point Ct,Burke,VA,22015,38.797391,-77.249788,https://www.realtor.com/realestateandhomes-det...
9,5243639661,2023-08-18 00:03:43.000000,for_sale,,condos,1982,229000,None,2,1,...,726,None,"10310 Rein Commons Ct Unit 3G Burke, VA 22015",10310 Rein Commons Ct Unit 3G,Burke,VA,22015,38.794856,-77.301015,https://www.realtor.com/realestateandhomes-det...


In [30]:
df.loc[df["id"] == '6963054695']

,id,list date,status,flags,home type,year built,price,hoa fee,beds,baths,interior sqft,lot sqft,address,addresss line,city,state,zipcode,latitude,longitude,url
0,6963054695,2023-10-06 15:58:37+00:00,for_sale,contingent,single_family,1968,799900,None,4,4,2528,10510,"6808 Houndmaster Rd Springfield, VA 22152",6808 Houndmaster Rd,Springfield,VA,22152,38.767167,-77.248466,https://www.realtor.com/realestateandhomes-det...


In [31]:
ds.loc[ds["id"] == '6963054695']

,id,list date,status,flags,home type,year built,price,hoa fee,beds,baths,interior sqft,lot sqft,address,addresss line,city,state,zipcode,latitude,longitude,url
20,6963054695,2023-10-06 15:58:37.000000,for_sale,contingent,single_family,1968,799900,None,4,4,2528,10510.0,"6808 Houndmaster Rd Springfield, VA 22152",6808 Houndmaster Rd,Springfield,VA,22152,38.767167,-77.248466,https://www.realtor.com/realestateandhomes-det...


In [37]:
new_records = df[df['id'] != ds['id']]

ValueError: Can only compare identically-labeled Series objects

In [34]:
new_records

,id,list date_x,status,flags_x,home type_x,year built_x,price_x,hoa fee_x,beds_x,baths_x,...,interior sqft_y,lot sqft_y,address_y,addresss line_y,city_y,state_y,zipcode_y,latitude_y,longitude_y,url_y


In [ ]:
merged_df.to_sql(name='House_Sale', con=engine, if_exists='APPEND', index=True)